In [2]:
import torch
from torchvision import transforms
from PIL import Image
import os
from torchvision import models,transforms
import torch.nn as nn

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Define a custom ResNet model with added dropout and a fully connected layer
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [10]:

# Function to load the model (assuming you have already trained it)
def load_model(model_path, num_classes):
    model = CustomResNet(num_classes=num_classes)  # Make sure the class matches the one you used for training
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()  # Set the model to evaluation mode
    return model

# Define a function to preprocess an image for prediction
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    
    image = Image.open(image_path).convert('RGB')  # Ensure the image is in RGB mode
    image = transform(image).unsqueeze(0)  # Add batch dimension (unsqueeze)
    
    return image

# Function to make a prediction
def predict_image(model, image_path, classes):
    image = preprocess_image(image_path)
    
    with torch.no_grad():  # Disable gradient computation for inference
        image = image.to(device)  # Move image to GPU if available
        outputs = model(image)
        _, predicted_class = torch.max(outputs, 1)
    
    predicted_class = predicted_class.item()  # Convert tensor to integer
    predicted_label = classes[predicted_class]
    
    return predicted_label

# Example usage
def main():
    model_path = 'newmodel.pth'  # Path to your trained model
    classes = ['Atelectasis', 'Covid19', 'Emphysema', 'Consolidation', 'Cardiomegaly', 'Infiltration', 'Nodule', 'Pneumonia', 'NORMAL', 'Pneumothorax']
    
    # Load the trained model
    model = load_model(model_path, num_classes=len(classes))
    model.to(device)  # Move model to GPU if available
    
    # Example image for prediction
    image_path = 'C:\\Users\\Acer\\Desktop\\Model\\image\\nodule.png'  # Provide path to the image you want to classify
    
    predicted_label = predict_image(model, image_path, classes)
    print(f"Predicted Class: {predicted_label}")

if __name__ == "__main__":
    main()


Predicted Class: Nodule
